In [1]:
import torch

from rvae.utils.paths import RESULTS_PATH

In [3]:
classification_results = torch.load(
    RESULTS_PATH / "predictions.pt", map_location=torch.device("cpu")
)

/tmp/ipykernel_30238/1200770556.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  classification_results = torch.load(


In [30]:
import numpy as np
from sklearn.metrics import f1_score
import torch


def compute_f1_each_model(predictions, targets):
    """
    Computes the F1 score for each Fashion MNIST class separately for each model,
    then computes the mean and standard deviation across models for each class.

    Args:
        predictions: A list of predictions from different models. Each element
            in the list should be a 1D NumPy array or a PyTorch tensor
            containing the predicted labels for a single model.
        targets: A 1D NumPy array or PyTorch tensor containing the true labels.

    Returns:
        f1_scores: A NumPy array of shape (num_models, num_classes) containing the
            F1 score for each class for each model.
        mean_f1: A NumPy array containing the mean F1 score across models for each class.
        std_f1: A NumPy array containing the standard deviation of the F1 scores across models for each class.
    """
    # Convert predictions and targets to NumPy arrays if necessary.
    preds_np = [
        p.detach().cpu().numpy() if isinstance(p, torch.Tensor) else p
        for p in predictions
    ]
    targets_np = (
        targets.detach().cpu().numpy() if isinstance(targets, torch.Tensor) else targets
    )

    f1_scores = []
    for p in preds_np:
        # Compute the per-class F1 score for this model.
        score = f1_score(targets_np, p, average=None)
        f1_scores.append(score)

    # Stack F1 scores into a (num_models, num_classes) array.
    f1_scores = np.vstack(f1_scores)

    # Compute the mean and standard deviation across models for each class.
    mean_f1 = f1_scores.mean(axis=0)
    std_f1 = f1_scores.std(axis=0)

    return f1_scores, mean_f1, std_f1

In [32]:
f1 = {}

for model_type in classification_results.keys():
    f1[model_type] = {}
    for latent_dim in classification_results[model_type].keys():
        targets = classification_results[model_type][latent_dim]["targets"]
        preds = classification_results[model_type][latent_dim]["predictions"]
        f1[model_type][latent_dim] = {}
        # f1s_ = []
        # for pred in preds:
        # print(f"DEBUG: {pred.shape=} | {targets.shape=}")
        # f1s_.append(
        #     compute_f1_per_class(
        #         predictions=pred.unsqueeze(1), targets=targets.unsqueeze(1)
        #     )
        # )
        _, f1[model_type][latent_dim]["mean"], f1[model_type][latent_dim]["std"] = (
            compute_f1_each_model(predictions=preds, targets=targets)
        )
        # = torch.stack(f1s_, dim=0).mean(dim=0)
        # compute_f1_per_class(
        # f1[model_type][latent_dim] =
        #     embeddings=embeddings,
        #     method_type="RVAE",
        #     latent_dim=latent_dim,
        #     subsample=None,
        # )

In [33]:
f1

{'VAE': {2: {'mean': array([0.7101347 , 0.92609256, 0.55994447, 0.67642076, 0.38041697,
          0.83592986, 0.16341775, 0.82145799, 0.94002201, 0.89061103]),
   'std': array([0.01010988, 0.00384236, 0.00934278, 0.01490139, 0.02176035,
          0.00506805, 0.04334456, 0.00292756, 0.00154986, 0.00438412])},
  5: {'mean': array([0.77110685, 0.96872954, 0.65665407, 0.823877  , 0.67234746,
          0.89441048, 0.43884465, 0.87829376, 0.94163189, 0.90188134]),
   'std': array([0.00458958, 0.00230181, 0.01010306, 0.00820092, 0.00947848,
          0.0036794 , 0.02339079, 0.00459297, 0.0035077 , 0.00230825])},
  10: {'mean': array([0.77488302, 0.96360167, 0.70119244, 0.82807512, 0.7447622 ,
          0.91804083, 0.52244228, 0.90231195, 0.95020883, 0.92045273]),
   'std': array([0.00322552, 0.00184557, 0.01553325, 0.00278505, 0.00593542,
          0.00379485, 0.01610617, 0.00135521, 0.00227852, 0.00214556])}},
 'RVAE': {2: {'mean': array([0.71239713, 0.92614889, 0.5613906 , 0.67660056, 0.366

In [43]:
flat_f1 = {
    (outer, inner, inner2): arr1
    for outer, inner_dict in f1.items()
    for inner, arr in inner_dict.items()
    for inner2, arr1 in arr.items()
}

In [44]:
flat_f1

{('VAE',
  2,
  'mean'): array([0.7101347 , 0.92609256, 0.55994447, 0.67642076, 0.38041697,
        0.83592986, 0.16341775, 0.82145799, 0.94002201, 0.89061103]),
 ('VAE',
  2,
  'std'): array([0.01010988, 0.00384236, 0.00934278, 0.01490139, 0.02176035,
        0.00506805, 0.04334456, 0.00292756, 0.00154986, 0.00438412]),
 ('VAE',
  5,
  'mean'): array([0.77110685, 0.96872954, 0.65665407, 0.823877  , 0.67234746,
        0.89441048, 0.43884465, 0.87829376, 0.94163189, 0.90188134]),
 ('VAE',
  5,
  'std'): array([0.00458958, 0.00230181, 0.01010306, 0.00820092, 0.00947848,
        0.0036794 , 0.02339079, 0.00459297, 0.0035077 , 0.00230825]),
 ('VAE',
  10,
  'mean'): array([0.77488302, 0.96360167, 0.70119244, 0.82807512, 0.7447622 ,
        0.91804083, 0.52244228, 0.90231195, 0.95020883, 0.92045273]),
 ('VAE',
  10,
  'std'): array([0.00322552, 0.00184557, 0.01553325, 0.00278505, 0.00593542,
        0.00379485, 0.01610617, 0.00135521, 0.00227852, 0.00214556]),
 ('RVAE',
  2,
  'mean'): arr

In [46]:
import pandas as pd

df = pd.DataFrame(data=flat_f1)

In [47]:
df.T

0         1         2         3         4         5  \
VAE  2  mean  0.710135  0.926093  0.559944  0.676421  0.380417  0.835930   
        std   0.010110  0.003842  0.009343  0.014901  0.021760  0.005068   
     5  mean  0.771107  0.968730  0.656654  0.823877  0.672347  0.894410   
        std   0.004590  0.002302  0.010103  0.008201  0.009478  0.003679   
     10 mean  0.774883  0.963602  0.701192  0.828075  0.744762  0.918041   
        std   0.003226  0.001846  0.015533  0.002785  0.005935  0.003795   
RVAE 2  mean  0.712397  0.926149  0.561391  0.676601  0.366243  0.828124   
        std   0.013942  0.002041  0.002662  0.017391  0.007280  0.007929   
     5  mean  0.769661  0.966254  0.655169  0.819338  0.670135  0.894848   
        std   0.003612  0.001329  0.007980  0.005484  0.017978  0.003987   
     10 mean  0.775877  0.964105  0.700487  0.828185  0.738791  0.920542   
        std   0.000690  0.001822  0.009865  0.003305  0.010388  0.002194   

                     6         7         8         9  
VAE  2  mean  0.163418  0.821458  0.940022  0.890611  
        std   0.043345  0.002928  0.001550  0.004384  
     5  mean  0.438845  0.878294  0.941632  0.901881  
        std   0.023391  0.004593  0.003508  0.002308  
     10 mean  0.522442  0.902312  0.950209  0.920453  
        std   0.016106  0.001355  0.002279  0.002146  
RVAE 2  mean  0.209999  0.821065  0.940262  0.889388  
        std   0.042195  0.004548  0.000978  0.003247  
     5  mean  0.421727  0.876305  0.941153  0.902575  
        std   0.022607  0.004697  0.005447  0.002248  
     10 mean  0.522853  0.903627  0.952616  0.922950  
        std   0.011226  0.003000  0.001997  0.001603

In [50]:
print(df.T.to_latex())

\begin{tabular}{lllrrrrrrrrrr}
\toprule
 &  &  & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
\midrule
\multirow[t]{6}{*}{VAE} & \multirow[t]{2}{*}{2} & mean & 0.710135 & 0.926093 & 0.559944 & 0.676421 & 0.380417 & 0.835930 & 0.163418 & 0.821458 & 0.940022 & 0.890611 \\
 &  & std & 0.010110 & 0.003842 & 0.009343 & 0.014901 & 0.021760 & 0.005068 & 0.043345 & 0.002928 & 0.001550 & 0.004384 \\
\cline{2-13}
 & \multirow[t]{2}{*}{5} & mean & 0.771107 & 0.968730 & 0.656654 & 0.823877 & 0.672347 & 0.894410 & 0.438845 & 0.878294 & 0.941632 & 0.901881 \\
 &  & std & 0.004590 & 0.002302 & 0.010103 & 0.008201 & 0.009478 & 0.003679 & 0.023391 & 0.004593 & 0.003508 & 0.002308 \\
\cline{2-13}
 & \multirow[t]{2}{*}{10} & mean & 0.774883 & 0.963602 & 0.701192 & 0.828075 & 0.744762 & 0.918041 & 0.522442 & 0.902312 & 0.950209 & 0.920453 \\
 &  & std & 0.003226 & 0.001846 & 0.015533 & 0.002785 & 0.005935 & 0.003795 & 0.016106 & 0.001355 & 0.002279 & 0.002146 \\
\cline{1-13} \cline{2-13}
\multirow[t]{6}{*}{RV